In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

In [2]:
import argparse
import sys
import tempfile

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.examples.tutorials.mnist import input_dataata

In [5]:
FLAGS = None

In [6]:
def deepNN(x):
    # Reshape input data.
    with tf.name_scope('reshape'):
        x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    # First convolution layer.
    with tf.name_scope('conv1'):
        W_conv1 = weight_variable([5,5,1,32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        
    ## Pooling layer.
    with tf.name_scope('pool1'):
        h_pool1 = max_pool_2x2(h_conv1)
    
    ## Convolution layer.
    with tf.name_scope('conv2'):
        W_conv2 = weight_variable([5,5,32,64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        
    ## Second pooling layer.
    with tf.name_scope('pool2'):
        h_pool2 = max_pool_2x2(h_conv2)
        
    ##fully connected layer.
    with tf.name_scope('fc1'):
        W_fc1 = weight_variable([7 * 7 * 64 , 1024])
        b_fc1 = bias_variable([1024])
        
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        
    ## Dropout layer.
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
        
    ## Fully connected layer 2.
    with tf.name_scope('fc2'):
        W_fc2 = weight_variable([1024, 10])
        b_fc2 = bias_variable([10])
        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
        
    return y_conv, keep_prob

        

In [7]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding= 'SAME')

In [8]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [9]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev= 0.1)
    return tf.Variable(initial)

In [10]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

In [32]:
def main(_):
  # Import data
    mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

  # Create the model
    x = tf.placeholder(tf.float32, [None, 784])

  # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 10])

  # Build the graph for the deep net
    y_conv, keep_prob = deepNN(x)

    with tf.name_scope('loss'):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                                            logits=y_conv)
    cross_entropy = tf.reduce_mean(cross_entropy)

    with tf.name_scope('adam_optimizer'):
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
    
    accuracy = tf.reduce_mean(correct_prediction)

    graph_location = tempfile.mkdtemp()
    print('Saving graph to: %s' % graph_location)
    train_writer = tf.summary.FileWriter(graph_location)
    train_writer.add_graph(tf.get_default_graph())

    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    
    for i in range(2000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print("Testing Accuracy is : ")
    print(sess.run(accuracy, feed_dict = {x : mnist.test.images, y_ : mnist.test.labels, keep_prob : 1.0}))


In [ ]:
tf.app.run(main = main)

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz
Saving graph to: C:\Users\lolla\AppData\Local\Temp\tmpo6_d0882
step 0, training accuracy 0.04
step 100, training accuracy 0.82
step 200, training accuracy 0.94
step 300, training accuracy 0.88
step 400, training accuracy 0.9
step 500, training accuracy 1
step 600, training accuracy 0.98
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 0.96
step 1100, training accuracy 0.94
step 1200, training accuracy 1
step 1300, training accuracy 0.98
step 1400, training accuracy 0.98
step 1500, training accuracy 0.96
step 1600, training accuracy 1
